# Coordle Backend

In [79]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain
import pymongo


import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [83]:
import inspect
import sys

print(__name__, sys.argv[0])

__main__ /home/jonavin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py


In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

In [4]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [5]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

ai_index = AI_Index(model.wv.most_similar, n_similars=1)
ai_index.build_from_df(
    df[:16],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 16/16 [00:00<00:00, 280.75it/s]


In [6]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index, RecursiveDescentParser, AI_Index

docs, scores, errmsgs = ai_index.search('virus')

n = 69
if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')


1wswi7us  Relationship of SARS-CoV to other pathogenic RNA viruses explored by t  39.6302
yy96yeu9  Viral Discovery and Sequence Recovery Using DNA Microarrays             15.6958
xqhn0vbp  Airborne rhinovirus detection and effect of ultraviolet irradiation on  10.8963
qj4dh6rg  Cloaked similarity between HIV-1 and SARS-CoV suggests an anti-SARS st  8.0116
5s6acr7m  The Virus That Changed My World                                         7.2854
le0ogx1s  A new recruit for the army of the men of death                          7.0304
0qaoam29  A double epidemic model for the SARS propagation                        5.1278
gi6uaa83  Discovering human history from stomach bacteria                         2.6177
ng4rrdte  Pro/con clinical debate: Steroids are a key component in the treatment  1.9475
fy4w7xz8  Association of HLA class I with severe acute respiratory syndrome coro  0.9052
1769ovyk  8th Annual Toronto Critical Care Medicine Symposium, 30 October–1 Nove  0.4316
kuybfc1y  Descript

In [7]:
ai_index.docmap['grace']

{1769ovyk}

In [8]:
with open('textfile.txt', 'w+') as f:
    uid = 'qzm9wgde'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping qzm9wgde


# Mongo

In [198]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index)

# ram_index = Index()
ram_index = AI_Index(model.wv.most_similar, 1)
ram_index.build_from_df(
    df[:128],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 128/128 [00:00<00:00, 359.41it/s]


In [162]:
model.wv.most_similar('corona')

[('coronavirus', 0.6643324494361877),
 ('coronovirus', 0.5752148032188416),
 ('coronaviruses', 0.5292266607284546),
 ('syncytial', 0.5095627307891846),
 ('rota', 0.5032534003257751),
 ('crownlike', 0.49367228150367737),
 ('paramyxo', 0.4860883057117462),
 ('flavi', 0.4837533235549927),
 ('herpes', 0.48291248083114624),
 ('picorna', 0.48181605339050293)]

In [244]:
import coordle_mongobackend as cm
reload(cm)

# mongoindex = cm.Index('coordle')
mongoindex = cm.AI_Index('coordle', model.wv.most_similar, 1)
mongoindex.drop_old_collections()
mongoindex.build_from_df(df, 'cord_uid', 'title', 'body_text', 
                         use_multiprocessing=True, workers=-1, verbose=True)

Text cleaning initilized on 16 workers


Initialized database index with size 36565
Inserting wordmap to database: 100%|█████████▉| 1635009/1635896 [00:32<00:00, 50399.89it/s] 


In [ ]:
docs, scores, errmsgs = ram_index.search('virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}   {score:.4f}')

In [245]:
%%time
docs, scores, errmsgs = mongoindex.search('virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc}   {score:.4f}')

deque(['virus', 'OR', 'viruses'])
s0jfadyp   0.0371
o1jhpqz1   0.0350
ipdyywbx   0.0339
j36mqpus   0.0334
ikitfme7   0.0332
9xbu4hnq   0.0331
wam2jl83   0.0329
okdiyk1m   0.0325
a0d50oni   0.0323
v6e6fmel   0.0308
n40be9ol   0.0298
i1ifocme   0.0297
n8zg6rsj   0.0296
93bvy38i   0.0287
ndt8f812   0.0285
xkmouavc   0.0277
db7hl770   0.0276
b6rmlpcv   0.0274
zubjxe8x   0.0273
5gsh9le2   0.0273
ei33h8ts   0.0272
oz4wvp5u   0.0268
lnch3qsq   0.0266
nl5kmmtd   0.0265
99pyker3   0.0265
hj8qanbb   0.0264
9xetsc2c   0.0263
w0ouin51   0.0262
d9kkhzej   0.0261
bzsc9289   0.0260
g2vcy9ea   0.0260
xekarlyg   0.0258
x4hdxx1t   0.0256
wk49ds8y   0.0253
bqx10w68   0.0251
ar372jke   0.0248
5meby3vc   0.0248
vgku5m5r   0.0247
7a0v0wfm   0.0245
7mojja4u   0.0244
q356npb7   0.0244
7pkbc785   0.0242
tg66h99q   0.0242
zp4svjd0   0.0241
1yoilmcw   0.0240
f8b4w3jk   0.0239
0dd4yzdh   0.0237
bhmnvq7c   0.0237
o6m5wz1z   0.0236
1s6dlcer   0.0236
r1mx2ewb   0.0234
ucg5468u   0.0233
v0uskzqh   0.0233
b5dw5mxf   0

j36vajdi   0.0099
2bsk6oh7   0.0099
5qym9dsf   0.0099
6y0xvp37   0.0099
er3mnkyf   0.0099
q4aeood2   0.0099
4k17voer   0.0099
ef8hqs4s   0.0099
7juagezq   0.0099
fxixiggj   0.0099
r68l7g2e   0.0099
2oc1yrzx   0.0099
7ydmp1pd   0.0099
fsx08qyd   0.0099
gf0bglm0   0.0099
qxp3mb9x   0.0099
jzfl4xuk   0.0099
m1jbpo5l   0.0099
slkp0w5b   0.0099
4r0f4g4b   0.0099
z637eh2z   0.0099
f15rvjok   0.0099
tfjba37y   0.0099
m0w8804y   0.0099
1jb6itlw   0.0099
q5j3vcfm   0.0099
26lvnqaq   0.0099
bupb4ooh   0.0099
tog8yxn5   0.0099
krvei97r   0.0099
l9cn0n99   0.0099
na5jqrpb   0.0099
l3gz0l4b   0.0098
p2kbjwpy   0.0098
d2re00r9   0.0098
e8sppjfz   0.0098
0w19i3xu   0.0098
8f8di4eu   0.0098
rb15hld7   0.0098
54s1fae3   0.0098
ljqn6f26   0.0098
2xy66mr5   0.0098
g2jfnxja   0.0098
91qwbslx   0.0098
ienq3aex   0.0098
so4qz9pk   0.0098
ckm0ol20   0.0098
c9ts2g7w   0.0098
se1kicnv   0.0098
2b420fz9   0.0098
g2o12kk3   0.0098
d5rza2f9   0.0098
662lpdx5   0.0098
agm4zbcx   0.0098
eilqme2l   0.0098
1vxl2468  

bu8vq65n   0.0071
hzaf52l4   0.0071
ee3p4lvu   0.0071
kcn67stj   0.0071
r2i1fogq   0.0071
oe8onyr9   0.0071
mttt1oai   0.0071
qi6qmgk2   0.0071
mkyxuhy2   0.0071
ky7xpock   0.0071
tcim3wrz   0.0071
1o1aopxh   0.0071
hygxwl2f   0.0070
8844fjic   0.0070
6mef38eo   0.0070
mkfwhv6v   0.0070
qeltdch7   0.0070
hmykmxv4   0.0070
z1nigtb5   0.0070
0t84eb53   0.0070
o0t5qd6a   0.0070
199yg0zj   0.0070
uixrz35f   0.0070
jol1lmh6   0.0070
yp9ofhi8   0.0070
0littefv   0.0070
vip4jtlv   0.0070
psbvp1b0   0.0070
9hvojcnk   0.0070
6jzkxubq   0.0070
54e285re   0.0070
zednsmnj   0.0070
ita2okbx   0.0070
pibq3iwh   0.0070
g4ntyddb   0.0070
chz0iwmj   0.0070
f1hcbsbb   0.0070
d5a90dnq   0.0070
um2khqhn   0.0070
iemjb7r9   0.0070
u4a8q76v   0.0070
ec7udsmw   0.0070
55b7xvmk   0.0070
fvj3cpie   0.0070
bcvedvgm   0.0070
qeojdpez   0.0070
1jvtb81p   0.0070
rk31pwoa   0.0070
mmg8jug2   0.0070
8z4aqfff   0.0070
muskjaw3   0.0070
q3fyo3rx   0.0070
g59whgep   0.0070
jwsi7asf   0.0070
zedd7ug4   0.0070
mgmbdfhe  

79qinq4t   0.0053
5t3ix35e   0.0053
t4x1p1rx   0.0053
awdly1ze   0.0053
bcry8yb7   0.0053
amv0los1   0.0053
s1hj89kj   0.0053
grofnanh   0.0053
okyq7rhn   0.0053
n1j5hzft   0.0053
tmkfju93   0.0053
bzpopzvr   0.0053
8n05ecrh   0.0053
s0ci3r8w   0.0053
8cbjhh1z   0.0053
ue7v0yab   0.0053
1a0fj900   0.0053
w5i752im   0.0053
wgigtgl9   0.0053
ah0dvnxv   0.0053
25tcxise   0.0053
clgzokb9   0.0053
euikj8fv   0.0053
cf336j6k   0.0053
y7jp49rj   0.0053
c8wp2m69   0.0053
avq3zwmc   0.0053
xipwslcb   0.0053
xj64bpgs   0.0053
ap9fkfgm   0.0053
yl973k2z   0.0053
ql2askp2   0.0053
wt73kx4g   0.0053
8wlpt4fa   0.0053
s1vyhijy   0.0053
nj1p4ehx   0.0053
at7oudll   0.0053
744r7gc3   0.0053
0bqpzwuq   0.0053
4k1w8ib1   0.0053
ifgef818   0.0053
7pdh7alh   0.0053
53l0yw16   0.0053
majpfxqn   0.0053
bbjmcdo5   0.0053
avjw4kx1   0.0053
0zchxz00   0.0053
e11tqf7s   0.0053
foxcukmn   0.0053
6om1y33o   0.0053
rt9xvfwy   0.0053
l2vvfro3   0.0053
0287r8k8   0.0053
kvfffe0b   0.0053
1h3wbhq4   0.0053
pl6gd3w4  

9jbhb8hh   0.0041
30duqivi   0.0041
5jm5c2lj   0.0041
t4l4an3t   0.0041
2gpsu1y4   0.0041
o6s5nw49   0.0041
jm7zqjsh   0.0041
tvjqnxy8   0.0041
rbf0k8oi   0.0041
xbrkytbs   0.0041
klywz85w   0.0041
nypzyx8x   0.0041
mdky6rdx   0.0041
oj86ua1j   0.0041
6cft36ad   0.0041
2em43ypc   0.0041
55ge7kmr   0.0041
czrea7kf   0.0041
w3c0on1t   0.0041
2s1xwxia   0.0041
c9vvhe8t   0.0041
vnvcqn4j   0.0041
cobroc5h   0.0041
vnwxqrha   0.0041
nekwhdln   0.0041
cuutlp1q   0.0041
aqtceqqo   0.0041
7giogyt9   0.0041
v5qb41c1   0.0041
bf3fre8r   0.0041
gcwtn3ei   0.0041
qjnetu71   0.0041
2sno82g7   0.0041
l951nirf   0.0041
iggo3ufb   0.0041
bsf8s4mh   0.0041
wcoq2dg1   0.0041
djkyhfsb   0.0041
o16kpx9s   0.0041
3ecnbqom   0.0041
4owfih6l   0.0041
yzxpapdd   0.0041
e2jp88n9   0.0041
x98194l0   0.0041
hxjh85n8   0.0041
795wuqz5   0.0041
e2zvhd8v   0.0041
m61s4y6u   0.0041
660ul4fb   0.0041
j6ao5d6x   0.0041
if1nh78w   0.0041
hjkx4k1a   0.0041
xaqua5ij   0.0041
wowuxw6o   0.0041
ssvxuu2g   0.0041
njfan7dp  

...

y5wbl6uo   0.0011
2ow90189   0.0011
ncb5r6gy   0.0011
x2ky94b0   0.0011
6r08hj06   0.0011
s471nmfj   0.0011
74i0v86g   0.0011
ejkrvze7   0.0011
bc30df2n   0.0011
nuioo1nh   0.0011
n38r8n70   0.0011
npc6dwk0   0.0011
hsspggp8   0.0011
7y5rq0nk   0.0011
aiorjsyh   0.0011
btn8v08m   0.0011
b5hj2phh   0.0011
nfmkfkqs   0.0011
6jczwxo6   0.0011
9n16ygxq   0.0011
3va5rvq5   0.0011
scn7b4c6   0.0011
g5daf8mc   0.0011
etx3chnt   0.0011
bdm7yvn1   0.0011
8m0hejek   0.0011
rbtyu4cu   0.0011
tfw4zqiy   0.0011
bgz1h20o   0.0011
ye2s0hzm   0.0011
3e1ji2mw   0.0011
a22s8xyz   0.0011
rn8xu82t   0.0011
ufy01x80   0.0011
byhdrgdc   0.0011
ds6x0yym   0.0011
qkznmdjy   0.0011
jac26dlz   0.0011
ul6baq0c   0.0011
r08k40ob   0.0011
7n8p9okf   0.0011
1y0pl31i   0.0011
8xob5edz   0.0011
54o2p36a   0.0011
0yfrux89   0.0011
mcyeyl4s   0.0011
7x2752j4   0.0011
m7zp4muo   0.0011
4taukqmf   0.0011
jslfu3qt   0.0011
42nam8cq   0.0011
2u7r00s1   0.0011
ylx751fv   0.0011
yuvjpziu   0.0011
rhphldyp   0.0011
hfdr3wnf  

7t5mm8og   0.0008
oxsdt3j1   0.0008
hge7ydrb   0.0008
fkea2rp8   0.0008
vdk2pxkk   0.0008
jzurpl1x   0.0008
hv83dd2g   0.0008
h0ordzm9   0.0008
kpubzqzw   0.0008
vvn4xciu   0.0008
rr8qet8c   0.0008
hix00vyd   0.0008
usu6laaq   0.0008
0ai8chbu   0.0008
is4yyblp   0.0008
0d9snlnv   0.0008
pcvq2e9q   0.0008
5wzco7nl   0.0008
1uf44jhc   0.0008
6r8v2hfx   0.0008
j10rdp7y   0.0008
qzw00q1s   0.0008
gugmzisp   0.0008
81ykqsm0   0.0008
7t7zgfvo   0.0008
0hfku8o4   0.0008
w0o8qt5k   0.0008
z29xsj1o   0.0008
jrm89ihe   0.0008
p6uqakzs   0.0008
ew6566bh   0.0008
r9dleue6   0.0008
2xrdv92m   0.0008
kd90e157   0.0008
8crvjzz4   0.0008
w7q5o8wc   0.0008
9zy94dd2   0.0008
l2yqrsq5   0.0008
b5myv6dc   0.0008
fvb6w11l   0.0008
ydssboge   0.0008
t85yioyl   0.0008
fexs1dm6   0.0008
bdi8v79t   0.0008
aprh5q3x   0.0008
7pim2j7z   0.0008
qqsmn72u   0.0008
qjsdndrs   0.0008
g1lajsp8   0.0008
vzk6qaq1   0.0008
lv8mll28   0.0008
pa9h6d0a   0.0008
la9svzml   0.0008
8m3hgain   0.0008
96qtlbeu   0.0008
hgkxyn0z  

jzke8fop   0.0005
wur4ss99   0.0005
h1s7b0qx   0.0005
zvu22n8f   0.0005
gnk3m0b8   0.0005
xzfo5jjq   0.0005
si9jhugj   0.0005
pl2d1yc0   0.0005
39vjafky   0.0005
dq2t1bbd   0.0005
yqe07wik   0.0005
uhrijlyz   0.0005
xftxcgeu   0.0005
x4i8dumy   0.0005
uvf5qzfd   0.0005
5dtksv1n   0.0005
qxk1wfk8   0.0005
ieobv7q8   0.0005
q6h1f69g   0.0005
fp0d03ze   0.0005
anbxo55z   0.0005
2ueghyvv   0.0005
toizrp89   0.0005
uo98qzm2   0.0005
pojx7xb9   0.0005
v1lwz9x6   0.0005
eas9u1mo   0.0005
89t3g9ty   0.0005
u56u9mjz   0.0005
ehd837s5   0.0005
62vp3zz5   0.0005
sbuxbclx   0.0005
a09bf50o   0.0005
ajfpghze   0.0005
a9xcab8t   0.0005
958630xh   0.0005
e36om23p   0.0005
xzionafe   0.0005
j4uftwit   0.0005
yye9soss   0.0005
zutavvrf   0.0005
gsqmkxjx   0.0005
1jwsq044   0.0005
xmbyxh4x   0.0005
h0bwyerl   0.0005
zlxqt7qy   0.0005
h122of8q   0.0005
h87vcmrd   0.0005
smlumvqc   0.0005
x46w59mf   0.0005
v471648u   0.0005
2ojxucuo   0.0005
wu2alj4u   0.0005
x5n4dyxp   0.0005
wllruqt6   0.0005
sbascdya  

z0uk7xgp   0.0003
883apt9n   0.0003
7xef0hdr   0.0003
wyhnquw6   0.0003
s37255t0   0.0003
7c1ldap5   0.0003
jopeo9gs   0.0003
p8e76bxr   0.0003
012ipcdr   0.0003
hrlfy1eq   0.0003
5s30ihv3   0.0003
yuvks61h   0.0003
afdmsbw2   0.0003
xc1ruvsz   0.0003
rn4zitcs   0.0003
2kkw9nwa   0.0003
zq2h0avw   0.0003
99lxmq4u   0.0003
ktet18wl   0.0003
d0ybqh8n   0.0003
n97kscde   0.0003
c2odlxwf   0.0003
g1kr7s11   0.0003
2meqe1vy   0.0003
0gwuvaj2   0.0003
zs9u8bp2   0.0003
tpoa0igy   0.0003
73hlwwdh   0.0003
q9nlwflu   0.0003
l8q3hqr3   0.0003
6a42tg1t   0.0003
ephlsr0k   0.0003
mn5l14lu   0.0003
vmsq5hhz   0.0003
ras60580   0.0003
wj47xeqj   0.0003
ha6j37r3   0.0003
xjbz9fw9   0.0003
q9o28iut   0.0003
fe061u76   0.0003
4ik3jxjy   0.0003
eujbxdqi   0.0003
hyoroqit   0.0003
9zmyojbu   0.0003
uq4eobf0   0.0003
vst954ef   0.0003
fk2mm8d3   0.0003
wwzxivzu   0.0003
aia9ajwx   0.0003
baby6742   0.0003
2dil8xf0   0.0003
frvhxuod   0.0003
g43ytph0   0.0003
ycdo72np   0.0003
7t87oshr   0.0003
h3dusudv  

1r65yam5   0.0002
66ulqu11   0.0002
uvuamh0p   0.0002
ygm0tc70   0.0002
c0uxx1g6   0.0002
1dygn0rw   0.0002
gjnyfzmo   0.0002
2p70g214   0.0002
znuqdzdp   0.0002
9kn22yf4   0.0002
h4uao6xs   0.0002
rzdjtd7j   0.0002
5pnjq7g2   0.0002
bfm4w84r   0.0002
0bozlx9t   0.0002
klp5iex3   0.0002
j580i7xm   0.0002
v2aapiqs   0.0002
cwuls6xs   0.0002
iwcync0o   0.0002
0lbbiq7u   0.0002
05jveocv   0.0002
z1i673yy   0.0002
0cxodo8j   0.0002
5wvbtxuy   0.0002
5283jsnu   0.0002
85eftmv3   0.0002
jnsis3gf   0.0002
lilcn688   0.0002
kyw4ptr2   0.0002
mjddkz95   0.0002
xv6j68wv   0.0002
p78vcu7y   0.0002
4e5ij2xp   0.0002
e2uzk4u1   0.0002
ho1y2orb   0.0002
6isk8jgj   0.0002
t4959z8b   0.0002
q0qhcfah   0.0002
69a8r235   0.0002
qvklgp5t   0.0002
5v4gor2v   0.0002
uqm7ndjt   0.0002
wschetbl   0.0002
6gjepq1h   0.0002
ast0ue6z   0.0002
cr54rn6t   0.0002
3ng0bo5a   0.0002
1dlohfbg   0.0002
rrecvc5n   0.0002
011k6mm0   0.0002
1ypgij14   0.0002
fz48wydz   0.0002
zso72hho   0.0002
7kplagj1   0.0002
5vgxg2cl  

In [13]:
wordmap = {word:[doc.uid for doc in docs] for word, docs in ai_index.docmap.items() if len(word) < 100}

In [14]:
A = pd.value_counts(['a','a','a','d','c','c','c','c','d'], sort=False)
A.index.values

array(['a', 'c', 'd'], dtype=object)

In [15]:
A = [[],0]
A[0].append(1)
A[1] += 1
A

[[1], 1]

In [16]:
client = pymongo.MongoClient()
db_test = client['coordle']
db_test.wordcounts.count_documents({})

2048

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<center><h2> Junkyard </h2></center>

In [17]:
raise ValueError

ValueError: 

Creating sets from dicts is faster than from lists

In [ ]:
from time import time 
from matplotlib import pyplot as plt 
import numpy as np

dict_time = []
list_time = []
ratios = []

xrange = np.arange(1000,1000000,1000)
B = set(list(range(100000)))

for j in xrange:
    a = list(range(j))

    t0=time()
    A = set(a)
    A | B
    t1_list = time()-t0
    list_time.append(t1_list)
    
    a = {i:None for i in range(j)}
    
    t0=time()
#     A = set(a)
    a.keys() | B
    t1_dict = time()-t0
    dict_time.append(t1_dict)

    ratios.append(t1_list/t1_dict)

In [ ]:
plt.plot(xrange[1:], dict_time[1:], label='dict')
plt.plot(xrange[1:], list_time[1:], label='list')
plt.legend()
plt.show()

In [ ]:
plt.plot(xrange[1:], ratios[1:])
plt.show()

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(rdp.parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    